# Лекция 9: Метрические методы (KNN) и Метод опорных векторов (SVM)

**Цель лекции:**
1.  Познакомиться с **метрическими методами** классификации на примере **K-ближайших соседей (KNN)**.
2.  Изучить один из самых мощных классических алгоритмов — **Метод опорных векторов (SVM)**.
3.  Разобрать математические основы, лежащие в основе этих алгоритмов.
4.  Научиться применять, настраивать и сравнивать эти модели на задачах бинарной и многоклассовой классификации с использованием Python и библиотеки Scikit-Learn.

## Часть 1: Метод K-ближайших соседей (K-Nearest Neighbors, KNN)

KNN относится к **метрическим алгоритмам**, в основе которых лежит **гипотеза компактности**. Она гласит, что объекты, принадлежащие к одному классу, в пространстве признаков расположены близко друг к другу, образуя "компактные" кластеры.

### 1.1. Интуиция и математические основы

Интуиция KNN очень проста: **класс нового объекта определяется тем, какой класс преобладает среди его ближайших соседей.**

![k=3](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/knn_illustration_k3.png)
![k=5](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/knn_illustration_k5.png)

Чтобы найти "ближайших" соседей, нам нужно уметь измерять расстояние. Для этого используются [**метрики**](http://lightcone.ru/manhattan/?ysclid=mhu3mmwtkq673949565)

#### Метрика Минковского
Это обобщенная метрика расстояния между двумя векторами $x$ и $x_i$ в n-мерном пространстве. Она имеет следующий вид:

$$ p(x, x_i) = \left( \sum_{j=1}^{n} \omega_j |x^j - x_i^j|^p \right)^{1/p} $$

где:
- $n$ — количество признаков (размерность пространства).
- $p > 0$ — параметр метрики, определяющий ее тип.
- $\omega_j$ — веса признаков. Они важны, когда признаки имеют разный масштаб или значимость. На практике, чтобы уравнять влияние признаков, данные почти всегда **масштабируют**.

![Разница между метриками расстояния](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/distance_metrics.png)

Два наиболее популярных частных случая метрики Минковского:

1.  **Евклидово расстояние (при p=2):** Привычное нам "прямое" расстояние.
$$ p(x, x_i) = \sqrt{ \sum_{j=1}^{n} (x^j - x_i^j)^2 } $$

2.  **Манхэттенское расстояние (при p=1):** "Расстояние городских кварталов", сумма модулей разностей координат.
$$ p(x, x_i) = \sum_{j=1}^{n} |x^j - x_i^j| $$

### 1.2. Алгоритм KNN

Для классификации нового объекта $x$ алгоритм выполняет следующие шаги:
1.  **Выбрать K** — количество соседей, на которое мы будем ориентироваться.
2.  **Вычислить расстояния** от $x$ до каждого объекта $x_i$ из обучающей выборки с помощью выбранной метрики.
3.  **Найти K соседей** — отобрать K объектов из обучающей выборки с минимальным расстоянием до $x$.
4.  **Определить класс** — класс объекта $x$ определяется классом, который является преобладающим среди найденных K соседей (мажоритарное голосование).

Математически, если $y^{(i)}$ — класс $i$-го соседа из $k$ ближайших, то класс нового объекта $a(x)$ находится так:

$$ a(x, X^l) = \arg\max_{y \in Y} \sum_{i=1}^{k} [y^{(i)} = y] $$

### 1.3. Преимущества и недостатки KNN

#### Преимущества:
*   **Простота и интуитивность:** Алгоритм легко понять и реализовать.
*   **Гибкость:** Легко адаптируется для задач регрессии (путем усреднения значений соседей, для этого в `scikit-learn` используется модуль **`KNeighborsRegressor`**).
*   **Нет этапа обучения:** KNN является "ленивым" алгоритмом. Он не строит модель, а просто запоминает всю обучающую выборку. Обучение происходит мгновенно.

#### Недостатки:
*   **Вычислительная сложность на этапе предсказания:** Для классификации одного нового объекта нужно рассчитать расстояние до всех объектов обучающей выборки, что может быть очень медленно на больших данных.
*   **Чувствительность к масштабу признаков:** Требует обязательного масштабирования данных.
*   **Чувствительность к выбору K:** Результат сильно зависит от этого гиперпараметра.
*   **Требовательность к памяти:** Нужно хранить всю обучающую выборку.

## Часть 2: Метод опорных векторов (Support Vector Machines, SVM)

SVM — один из самых мощных и популярных алгоритмов классификации. Его основная идея — найти не просто разделяющую границу, а **оптимальную разделяющую гиперплоскость**.

### 2.1. Интуиция и математические основы

#### Гиперплоскость
Гиперплоскость — это обобщение прямой (для 2D) и плоскости (для 3D) на пространство любой размерности. Это подпространство, размерность которого на единицу меньше исходного.

![Примеры гиперплоскостей в разных размерностях](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/hyperplanes.png)

Уравнение гиперплоскости задается в виде:

$$ w^T x - b = 0 \quad \text{или} \quad \langle w, x \rangle - b = 0 $$

где:
- $w$ — вектор весов (нормаль к гиперплоскости), который задает ее ориентацию.
- $x$ — вектор признаков объекта.
- $b$ — смещение (bias), которое определяет положение гиперплоскости.

#### Максимизация зазора (Margin)
SVM ищет такую гиперплоскость, которая находится на максимальном удалении от ближайших объектов каждого класса. Этот зазор называется **margin**. Объекты, лежащие на границах этого зазора, называются **опорными векторами** (support vectors), так как именно они "поддерживают" гиперплоскость и определяют ее положение.

![Оптимальная разделяющая гиперплоскость](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/оптимальная_разделяющая_гиперплоскость.png)


Задача SVM для **линейно разделимого** случая сводится к задаче квадратичного программирования: мы хотим **минимизировать** норму вектора весов (что эквивалентно максимизации зазора), при условии, что все точки классифицированы правильно.

$$ \frac{1}{2} ||w||^2 \rightarrow \min_{w,b} $$
При ограничении:
$$ y_i(\langle w, x_i \rangle - b) \ge 1, \quad i=1, \dots, l $$

**Давайте разберем эту формулу:**
1.  **Выражение $\langle w, x_i \rangle - b$** — это "счет" для точки $x_i$. Его знак показывает, с какой стороны от центральной линии находится точка.
2.  **Умножение на $y_i$** (метка класса, `+1` или `-1`) — это трюк. Если точка классифицирована правильно, результат этого умножения всегда будет положительным.
3.  **Требование $\ge 1$** — это самое главное. Мы требуем, чтобы каждая точка находилась не просто на правильной стороне от центра (`> 0`), а на своей границе зазора (`= 1`) или еще дальше (`> 1`). Это и создает "пустую зону" между классами, ширину которой мы и стремимся максимизировать.

### 2.2. Что такое "Опорные Векторы"?

Название "Метод Опорных Векторов" говорит само за себя. **Опорные векторы** — это ключевые точки из обучающей выборки, которые определяют положение и ориентацию оптимальной разделяющей гиперплоскости.

**Концепция:** Представьте, что вы разделяете два вида монет на столе линейкой. Чтобы зазор был максимальным, вы будете двигать линейку, пока она не упрется в несколько монет с обеих сторон. Эти монеты, которые "подпирают" линейку, и есть **опорные векторы**. Все остальные монеты, которые находятся далеко, на положение линейки не влияют.

**Определение:**
1.  **В идеальном случае (Hard Margin):** Опорными векторами являются только те точки, которые лежат **ровно на границах зазора** (где $ y_i(\langle w, x_i \rangle - b) = 1 $).
2.  **В реальном случае (Soft Margin):** Опорными векторами являются **все точки-нарушители**: те, что лежат на границе зазора, внутри него или были классифицированы неверно.

**Почему это важно?**
- **Эффективность:** После обучения модель SVM хранит в памяти **только опорные векторы**, а не всю обучающую выборку. Для предсказания нового объекта она сравнивает его **только с этими опорными векторами**, что делает SVM очень быстрым на этапе предсказания.

#### Мягкий зазор (Soft Margin) и неразделимые данные
В реальности данные часто нельзя разделить идеально. Для таких случаев вводится концепция "мягкого зазора". Мы разрешаем некоторым точкам нарушать границы зазора или даже быть неверно классифицированными. 

![Пример мягкого зазора (Soft Margin)](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/пример_мягкого_зазора_(soft_margin).png)

Для этого вводятся **слабые переменные** (slack variables) $\xi_i \ge 0$, которые показывают, насколько $i$-й объект нарушает границу.

Оптимизационная задача усложняется: теперь мы минимизируем не только норму вектора весов, но и суммарную ошибку (сумму $\xi_i$):

$$ \frac{1}{2} ||w||^2 + C \sum_{i=1}^{l} \xi_i \rightarrow \min_{w,b,\xi} $$
Ограничение ослабляется:
$$ y_i(\langle w, x_i \rangle - b) \ge 1 - \xi_i, \quad \xi_i \ge 0, \quad i=1, \dots, l $$

*   Если $\xi_i = 0$, точка находится за пределами зазора (все в порядке).
*   Если $0 < \xi_i \le 1$, точка попала внутрь зазора, но классифицирована верно.
*   Если $\xi_i > 1$, точка была классифицирована неверно.

Модель пытается одновременно **максимизировать зазор** (минимизировать $||w||^2$) и **минимизировать суммарную ошибку** (минимизировать $\sum \xi_i$). Баланс между этими двумя целями контролируется **гиперпараметром `C`**.

**Гиперпараметр `C`** — это параметр регуляризации. Он контролирует баланс между максимизацией зазора и минимизацией количества ошибок:
- **Маленький `C`**: Мы предпочитаем широкий зазор, даже если это приведет к большему числу ошибок на обучающей выборке. Модель более простая, меньше склонна к переобучению (высокое смещение, низкая дисперсия).
- **Большой `C`**: Мы сильно штрафуем за ошибки, поэтому модель пытается классифицировать каждую точку правильно, даже ценой сужения зазора. Модель более сложная и склонна к переобучению (низкое смещение, высокая дисперсия).

#### Трюк с ядром (Kernel Trick)

Что делать, если данные неразделимы даже с мягким зазором (например, один класс находится внутри другого)? Идея состоит в том, чтобы перевести данные в пространство признаков более высокой размерности, где они, возможно, станут линейно разделимыми.

![Визуализация трюка с ядром](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/kernel_trick_visualization.png)



#### Как SVM делает предсказания (Двойственная форма)

Оказывается, формулу для предсказания SVM можно записать в виде, который зависит не от вектора $w$, а напрямую от опорных векторов:

$$ \text{предсказание}(x) = \text{sign} \left( \sum_{i \in SV} \alpha_i y_i \langle x_i, x \rangle - b \right) $$

где:
- $x$ — новый объект, который мы хотим классифицировать.
- $x_i$ — $i$-й **опорный вектор**, который модель запомнила во время обучения.
- $\alpha_i$ и $y_i$ — вес и класс этого опорного вектора.
- $\langle x_i, x \rangle$ — скалярное произведение, которое измеряет "схожесть" нового объекта с $i$-м опорным вектором.

**Простыми словами:** чтобы классифицировать новый объект, SVM измеряет его схожесть с каждым из запомненных опорных векторов, суммирует эти схожести с весами и на основе итогового знака выносит вердикт.

#### Что такое ядро?

**Ядро — это вычислительный трюк.** Это функция $K(x_i, x)$, которая позволяет нам получить результат скалярного произведения в новом, очень сложном пространстве, **не выполняя само преобразование в это пространство.**

Мы просто заменяем $\langle x_i, x \rangle$ на $K(x_i, x)$ в формуле предсказания:

$$ \text{предсказание}(x) = \text{sign} \left( \sum_{i \in SV} \alpha_i y_i K(x_i, x) - b \right) $$

**Популярные ядра:**
1.  **Линейное:** $K(x_i, x) = \langle x_i, x \rangle$.
2.  **Полиномиальное:** $K(x_i, x_j) = (\gamma \langle x_i, x_j \rangle + r)^d$. Гиперпараметры: степень $d$, коэффициент $\gamma$, свободный член $r$.
3.  **Радиальная базисная функция (RBF):** $K(x_i, x) = \exp(-\gamma ||x_i - x||^2)$. Вычисляет "схожесть" на основе Гауссовой функции. Контролируется параметром `gamma`.  Маленькая `gamma` означает большое влияние (гладкая граница), большая `gamma` — локальное влияние (очень сложная, извилистая граница).

### 2.3. Демонстрация 'Трюка с ядром' на практике

Давайте на простом, наглядном примере посмотрим, почему ядра так важны. Мы создадим синтетический набор данных, который невозможно разделить прямой линией.

In [ ]:
from sklearn.datasets import make_circles
import matplotlib.pyplot as plt
import numpy as np

# Создаем данные: один класс (0) в виде круга внутри другого класса (1)
X, y = make_circles(n_samples=100, noise=0.1, factor=0.5, random_state=42)

# Визуализируем
plt.figure(figsize=(8, 6))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='winter')
plt.title('Линейно неразделимые данные (круги)')
plt.show()

Как мы видим, разделить эти данные одной прямой линией невозможно. Попробуем обучить SVM с двумя разными ядрами.

**Попытка №1: Линейное ядро**

In [ ]:
from sklearn.svm import SVC

# Вспомогательная функция для отрисовки разделяющей поверхности
def plot_decision_boundary(model, X, y):
    x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
    y_min, y_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                           np.linspace(y_min, y_max, 100))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, alpha=0.3, cmap='winter')
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap='winter', edgecolors='k')
    plt.title(f'Ядро: {model.kernel}')

# Обучаем модель с линейным ядром
linear_svm = SVC(kernel='linear').fit(X, y)

# Визуализируем результат
plt.figure(figsize=(8, 6))
plot_decision_boundary(linear_svm, X, y)
plt.show()
print(f"Точность линейного SVM: {linear_svm.score(X,y):.2f}")

Линейное ядро потерпело неудачу. Модель провела прямую линию, которая плохо разделяет классы, и точность очень низкая.

**Попытка №2: Нелинейное ядро RBF**

In [ ]:
# Обучаем модель с ядром RBF (используется по умолчанию)
rbf_svm = SVC(kernel='rbf', C=1, gamma='auto').fit(X, y)

# Визуализируем результат
plt.figure(figsize=(8, 6))
plot_decision_boundary(rbf_svm, X, y)
plt.show()
print(f"Точность RBF SVM: {rbf_svm.score(X,y):.2f}")

**Вывод:** Ядро RBF успешно справилось с задачей! Оно построило нелинейную, круговую границу, которая идеально разделила классы, и точность стала 100%. Этот пример наглядно демонстрирует, как "трюк с ядром" позволяет SVM решать сложные, нелинейные задачи, просто изменив один гиперпараметр.

### 2.4. Преимущества и недостатки SVM

#### Преимущества:
*   **Эффективность в пространствах высокой размерности:** Отлично работает, когда признаков много.
*   **Эффективность по памяти:** Использует только часть обучающих точек для построения модели (опорные векторы).
*   **Гибкость:** Благодаря ядрам может строить очень сложные нелинейные разделяющие поверхности.
*   **Имеет прочное математическое обоснование** в виде задачи выпуклой оптимизации, что гарантирует нахождение глобального минимума.

#### Недостатки:
*   **Чувствительность к выбору ядра и его гиперпараметров (`C`, `gamma`):** Требует тщательного подбора с помощью `GridSearchCV`.
*   **Вычислительная сложность:** Обучение может быть долгим на очень больших наборах данных.
*   **Сложность интерпретации:** Модель с нелинейным ядром является "черным ящиком", ее решения сложно интерпретировать.

### 2.5. Применение SVM для задач регрессии (SVR)

Метод опорных векторов можно элегантно адаптировать и для решения задач регрессии. Этот подход называется **Support Vector Regression (SVR)**.

#### Интуиция: от "максимального зазора" к "максимально широкой улице"

Вспомним, что в классификации SVM искал гиперплоскость, которая бы имела **максимальный зазор** между классами. В регрессии классов нет, у нас есть непрерывная целевая переменная.

Основная идея SVR — сделать обратное. Вместо того чтобы раздвигать точки, мы хотим найти такую функцию (гиперплоскость), вокруг которой можно построить "улицу" или "коридор" с как можно большим количеством точек **внутри** этого коридора.

*   **Цель:** Провести линию регрессии так, чтобы она была как можно ближе к большинству точек.
*   **"Улица" (Коридор):** Мы определяем коридор шириной `2ε` (два эпсилон) вокруг нашей линии регрессии.
*   **Правило:** Мы **не штрафуем** модель за ошибки, если точки данных попадают **внутрь** этого коридора. Штраф начисляется только за те точки, которые оказались **снаружи**.

![Принцип работы SVR](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/svr_illustration.png)

#### Математическая постановка

Задача SVR, как и у SVM, сводится к задаче выпуклой оптимизации. Мы снова хотим минимизировать норму вектора весов $||w||^2$ (что делает модель более "гладкой"), но при других ограничениях.

Мы хотим найти такую функцию $f(x) = \langle w, x \rangle + b$, которая имеет наименьшее отклонение от реальных значений $y_i$, "прощая" отклонения меньше, чем $\epsilon$.

Это приводит к следующей задаче минимизации ("примальная" форма):

$$ \frac{1}{2} ||w||^2 + C \sum_{i=1}^{l} (\xi_i + \xi_i^*) \rightarrow \min_{w,b,\xi,\xi^*} $$

При ограничениях:
$$ y_i - (\langle w, x_i \rangle + b) \le \epsilon + \xi_i \quad (\text{для точек выше коридора}) $$
$$ (\langle w, x_i \rangle + b) - y_i \le \epsilon + \xi_i^* \quad (\text{для точек ниже коридора}) $$
$$ \xi_i, \xi_i^* \ge 0 $$

*   $\epsilon$ (эпсилон) — ширина "зоны нечувствительности" в одну сторону от линии.
*   $\xi_i$ и $\xi_i^*$ — слабые переменные, измеряющие, **насколько** точка $x_i$ "вылетела" за пределы коридора вверх или вниз.
*   `C` — параметр регуляризации, контролирующий баланс между "гладкостью" модели и количеством ошибок.

#### Двойственная форма и роль опорных векторов в SVR

Как и в классификации, для практических вычислений (и использования ядер) используется "двойственная" форма. Итоговая формула для предсказания выглядит так:

$$ \text{предсказание}(x) = \left( \sum_{i \in SV} (\alpha_i - \alpha_i^*) K(x_i, x) \right) + b $$

**Давайте расшифруем эту ключевую формулу:**
*   **$K(x_i, x)$** — это ядро, измеряющее "схожесть" нового объекта $x$ с опорным вектором $x_i$.
*   **$SV$** — это множество **опорных векторов**, то есть только тех точек, которые лежат на границе $\epsilon$-коридора или за его пределами.
*   **$\alpha_i$ и $\alpha_i^*$** — это **множители Лагранжа**, которые можно интуитивно представить как "силы", с которыми опорные векторы "тянут" на себя линию регрессии.
    *   **$\alpha_i > 0$** (и $\alpha_i^*=0$) для точек, оказавшихся **выше** коридора. Они тянут линию **вверх**.
    *   **$\alpha_i^* > 0$** (и $\alpha_i=0$) для точек, оказавшихся **ниже** коридора. Они тянут линию **вниз**.
    *   Для всех точек **внутри** коридора и $\alpha_i=0$, и $\alpha_i^*=0$. Они **не влияют** на итоговую модель.
*   Выражение **$(\alpha_i - \alpha_i^*)$** — это итоговый **"вес"** $i$-го опорного вектора в модели. Он положителен, если точка тянет вверх, и отрицателен, если точка тянет вниз.

**Простыми словами:** итоговая регрессионная кривая строится как взвешенная сумма "схожестей" нового объекта со всеми точками-нарушителями (опорными векторами).

#### Реализация в Scikit-Learn

В `scikit-learn` есть несколько классов для регрессии методом опорных векторов:

1.  **`sklearn.svm.SVR`**:
    *   Самая универсальная реализация, поддерживающая все виды ядер (`linear`, `poly`, `rbf`).
    *   **Ключевые гиперпараметры:**
        *   `kernel`: Тип ядра. По умолчанию `'rbf'`.
        *   `C`: Параметр регуляризации.
        *   `gamma`: Коэффициент для `rbf` и `poly` ядер.
        *   `epsilon`: Ширина $\epsilon$-коридора.

2.  **`sklearn.svm.LinearSVR`**:
    *   Специализированная и быстрая реализация **только для линейного ядра**.
    *   Значительно быстрее, чем `SVR(kernel='linear')` на больших данных.
    *   **Когда использовать:** Когда зависимость в данных линейная, и важна скорость.

3.  **`sklearn.svm.NuSVR`**:
    *   Альтернативная реализация, использующая параметр `nu` вместо `C`.
    *   `nu` (от 0 до 1) контролирует долю опорных векторов.

**Основное отличие `SVR` от `LinearSVR`** — `SVR` позволяет строить нелинейные модели благодаря ядрам, в то время как `LinearSVR` оптимизирован исключительно для линейных задач.

## Часть 3: Схема программирования модели (Workflow)

Независимо от выбранного алгоритма (KNN, SVM или другой), процесс его применения в коде следует стандартной схеме. Для этого мы используем мощные библиотеки Python: `pandas` для работы с данными, `matplotlib` и `seaborn` для визуализации, и `scikit-learn` для машинного обучения.

#### Шаг 1: Загрузка и подготовка данных
*   **Что делаем:** Загружаем данные (например, из CSV) и проводим разведочный анализ (EDA).
*   **Библиотеки:** `import pandas as pd`, `import seaborn as sns`
*   **Пример:** `df = pd.read_csv('data.csv')`, `sns.countplot(df['target'])`

#### Шаг 2: Определение признаков (X) и цели (y)
*   **Что делаем:** Разделяем наш датафрейм на матрицу признаков `X` и вектор целевой переменной `y`.
*   **Пример:** `X = df.drop('target', axis=1)`, `y = df['target']`

#### Шаг 3: Разделение на обучающую и тестовую выборки
*   **Что делаем:** Отделяем часть данных, на которой модель не будет обучаться, чтобы честно оценить ее качество.
*   **Библиотеки:** `from sklearn.model_selection import train_test_split`
*   **Пример:** `X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)`

#### Шаг 4: Масштабирование признаков
*   **Что делаем:** Приводим все признаки к единому масштабу. **Критически важно для KNN и SVM!**
*   **Библиотеки:** `from sklearn.preprocessing import StandardScaler`
*   **Пример:** `scaler = StandardScaler()`, `X_train_scaled = scaler.fit_transform(X_train)`, `X_test_scaled = scaler.transform(X_test)`
*   **Важно:** Метод `fit` (вычисление среднего и стандартного отклонения) вызывается **только на обучающих данных**, чтобы избежать утечки информации из теста!

#### Шаг 5: Создание и обучение модели
*   **Что делаем:** Создаем экземпляр модели и обучаем его на масштабированных обучающих данных.
*   **Библиотеки:** `from sklearn.neighbors import KNeighborsClassifier`, `from sklearn.svm import SVC`
*   **Пример:** `model = SVC(C=1.0, kernel='rbf')`, `model.fit(X_train_scaled, y_train)`

#### Шаг 6: Предсказание и оценка качества
*   **Что делаем:** Делаем предсказания на тестовых данных и сравниваем их с реальными значениями.
*   **Библиотеки:** `from sklearn.metrics import classification_report, confusion_matrix`
*   **Пример:** `predictions = model.predict(X_test_scaled)`, `print(classification_report(y_test, predictions))`

#### Оптимизация: Pipeline и GridSearchCV
Чтобы автоматизировать шаги 4-6 и найти лучшие гиперпараметры, используются `Pipeline` (объединяет шаги в конвейер) и `GridSearchCV` (перебирает параметры по сетке).

## Часть 4: Практические примеры на Python

In [ ]:
# Импортируем все необходимые библиотеки
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import load_wine

# Настройки для лучшей визуализации
sns.set_style('whitegrid')

### Пример 1: Бинарная классификация (Подделка вина)

**Задача:** На основе химического анализа определить, является ли вино настоящим (`Legit`) или поддельным (`Fraud`).

In [ ]:
# Загружаем данные
df_fraud = pd.read_csv('https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/data/wine_fraud.csv')

# EDA
print('Информация о датасете:')
df_fraud.info()
print('\nБаланс классов:')
print(df_fraud['quality'].value_counts())
sns.countplot(x='quality', data=df_fraud)
plt.title('Баланс классов в датасете о вине')
plt.show()

# Подготовка данных
X = df_fraud.drop('quality', axis=1)
# Конвертируем 'Legit'/'Fraud' в 0/1 для удобства
y = df_fraud['quality'].map({'Legit': 0, 'Fraud': 1})

# Обработка категориального признака 'type'
X = pd.get_dummies(X, columns=['type'], drop_first=True)

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)

#### Модель KNN для бинарной классификации

In [ ]:
# Создаем Pipeline: сначала масштабирование, потом модель KNN
knn_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

# Задаем сетку параметров для перебора: ищем лучшее k
k_values = list(range(1, 20))
param_grid_knn = {'knn__n_neighbors': k_values}

# Создаем и обучаем GridSearchCV
grid_knn = GridSearchCV(knn_pipe, param_grid_knn, cv=5, scoring='accuracy')
grid_knn.fit(X_train, y_train)

print(f"Лучший параметр для KNN: {grid_knn.best_params_}")

# Оценка модели KNN
knn_preds = grid_knn.predict(X_test)
print("\n--- Отчет по качеству модели KNN ---")
print(classification_report(y_test, knn_preds))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
 
cm = confusion_matrix(y_test, knn_preds)
 
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Legit', 'Fraud'], yticklabels=['Legit', 'Fraud'])
plt.xlabel('Предсказанный класс')
plt.ylabel('Истинный класс')
plt.title('Матрица ошибок для модели KNN')
plt.savefig("confusion_matrix_heatmap.png")
plt.show()

#### Модель SVM для бинарной классификации

In [ ]:
# Создаем Pipeline для SVM
svm_pipe = Pipeline([
    ('scaler', StandardScaler()),
    # Учитываем дисбаланс классов с помощью class_weight='balanced'
    ('svm', SVC(class_weight='balanced'))
])

# Сетка параметров для SVM. C и gamma - ключевые параметры RBF ядра
param_grid_svm = {
    'svm__C': [0.1, 1, 10],
    'svm__gamma': ['scale', 'auto', 0.1]
}

# Создаем и обучаем GridSearchCV
grid_svm = GridSearchCV(svm_pipe, param_grid_svm, cv=5, scoring='accuracy')
grid_svm.fit(X_train, y_train)

print(f"Лучшие параметры для SVM: {grid_svm.best_params_}")

# Оценка модели SVM
svm_preds = grid_svm.predict(X_test)
print("\n--- Отчет по качеству модели SVM ---")
print(classification_report(y_test, svm_preds))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
 
cm = confusion_matrix(y_test, svm_preds)
 
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Legit', 'Fraud'], yticklabels=['Legit', 'Fraud'])
plt.xlabel('Предсказанный класс')
plt.ylabel('Истинный класс')
plt.title('Матрица ошибок для модели SVM')
plt.savefig("confusion_matrix_heatmap.png")
plt.show()

#### Сравнение KNN и SVM на задаче бинарной классификации

**Выводы:**
1.  **Метрики:** Сравните отчеты `classification_report`. Какая модель дает более высокий `f1-score` для класса `Fraud` (метка 1)? В задачах поиска мошенничества или дефектов часто важнее **полнота (recall)**, чтобы найти как можно больше случаев мошенничества, даже ценой ложных срабатываний.
2.  **Производительность:** На этом датасете SVM, скорее всего, покажет себя лучше. Благодаря своей способности строить сложную границу с максимальным зазором, он эффективнее разделяет классы. 
3.  **Сложность:** Настройка SVM сложнее из-за большего количества гиперпараметров (`C`, `gamma`, тип ядра), но `GridSearchCV` автоматизирует этот процесс.

### Пример 2: Многоклассовая классификация (Сорта вин)

**Задача:** Классифицировать вино по одному из трех сортов на основе 13 химических признаков.

#### Как алгоритмы решают многоклассовую задачу?
-   **KNN:** Естественным образом. Он просто находит K ближайших соседей и смотрит, какой из 3+ классов преобладает.
-   **SVM:** SVM по своей природе бинарный классификатор. Для решения многоклассовой задачи он использует одну из стратегий:
    -   **One-vs-Rest (OvR):** Обучается N классификаторов, где N — число классов. Каждый классификатор учится отличать "свой" класс от всех остальных (`Rest`).
    -   **One-vs-One (OvO):** Обучается N * (N-1) / 2 классификаторов для каждой возможной пары классов. Новый объект классифицируется тем классом, который "победил" в большинстве "дуэлей". **Scikit-learn использует эту стратегию по умолчанию для `SVC`**, так как она часто эффективнее.


In [ ]:
# Загрузка данных из sklearn
wine_data = load_wine()
X = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)
y = pd.Series(wine_data.target)

# EDA
print("Размеры датасета сортов вин:", X.shape)
print("\nБаланс классов:")
print(y.value_counts())

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)

#### Модель KNN для многоклассовой классификации

In [ ]:
# Используем тот же Pipeline, что и раньше
grid_knn_multi = GridSearchCV(knn_pipe, param_grid_knn, cv=5, scoring='accuracy')
grid_knn_multi.fit(X_train, y_train)

print(f"Лучший параметр для KNN: {grid_knn_multi.best_params_}")

# Оценка
knn_preds_multi = grid_knn_multi.predict(X_test)
print("\n--- Отчет по качеству модели KNN (многоклассовая) ---")
print(classification_report(y_test, knn_preds_multi))

print("Матрица ошибок:")
print(confusion_matrix(y_test, knn_preds_multi))

#### Модель SVM для многоклассовой классификации

In [ ]:
# Pipeline для SVM без class_weight, т.к. классы сбалансированы
svm_pipe_multi = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

param_grid_svm_multi = {
    'svm__C': [0.1, 1, 10, 100],
    'svm__gamma': ['scale', 'auto', 0.1, 0.01]
}

grid_svm_multi = GridSearchCV(svm_pipe_multi, param_grid_svm_multi, cv=5, scoring='accuracy')
grid_svm_multi.fit(X_train, y_train)

print(f"Лучшие параметры для SVM: {grid_svm_multi.best_params_}")

svm_preds_multi = grid_svm_multi.predict(X_test)
print("\n--- Отчет по качеству модели SVM (многоклассовая) ---")
print(classification_report(y_test, svm_preds_multi))

print("Матрица ошибок:")
print(confusion_matrix(y_test, svm_preds_multi))

#### Сравнение на задаче многоклассовой классификации

**Выводы:**
1.  **Интерпретация метрик:** В отчете теперь есть строки для каждого из трех классов (0, 1, 2) и усредненные значения (`macro avg`, `weighted avg`).
2.  **Матрица ошибок:** Теперь это матрица 3x3. Элемент на пересечении строки `i` и столбца `j` показывает, сколько объектов реального класса `i` было предсказано как класс `j`. Диагональные элементы — это правильно классифицированные объекты. Внедиагональные — ошибки. Анализ этой матрицы помогает понять, какие классы модель путает между собой.
3.  **Результат:** На этом классическом датасете обе модели, скорее всего, покажут очень высокие результаты, близкие к 100% точности, но SVM с правильно подобранными параметрами часто оказывается немного точнее.

## Итоговое заключение

Мы рассмотрели два мощных, но очень разных по своей природе алгоритма классификации.

| Характеристика | K-ближайших соседей (KNN) | Метод опорных векторов (SVM) |
| :--- | :--- | :--- |
| **Основная идея** | Классификация по большинству голосов соседей | Поиск оптимальной разделяющей гиперплоскости |
| **Тип модели** | Метрический, "ленивый" (не строит модель) | Геометрический, строит явную модель |
| **Ключевые параметры**| `n_neighbors` (количество соседей) | `C` (регуляризация), `kernel`, `gamma` (для RBF) |
| **Требования** | **Обязательное** масштабирование данных | **Обязательное** масштабирование данных |
| **Лучше всего подходит** | Для быстрых прототипов, когда данные "чистые" и хорошо сгруппированы | Для сложных задач с нелинейными границами, в пространствах высокой размерности |
| **Интерпретируемость** | Высокая (можно посмотреть на соседей) | Низкая (особенно с нелинейными ядрами) |